In [1]:
import sys

sys.path.append('../../..')

In [9]:
%pdb

Automatic pdb calling has been turned OFF


In [15]:
get_devices(list(range(11)))

[[['0', '1'], ['2', '3']], [['4', '5'], ['6', '7']], [['8', '9'], ['10', '0']]]

In [23]:
from batchflow.research import Research

def power(a=2, b=3):
    return a ** b

research = Research().add_callable(power, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  2.32it/s]


,id,iteration,name,value
0,b8c9f03199415848,0,power,8


In [3]:
research = Research().add_callable(power, a=3, b=2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


,id,iteration,name,value
0,b8c9f03130430383,0,power,9


In [4]:
research = Research().add_callable(power, a=3, b=2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  3.49it/s]


,id,iteration,name,value
0,b8c9f03130430383,0,power,9


In [5]:
research = Research().power(3, 2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  2.76it/s]


,id,iteration,name,value
0,b8c9f03130430383,0,power,9


In [6]:
from batchflow.research import Domain, EC

domain = Domain({'a': [2, 3], 'b': [2, 3, 4]})

research = Research(domain=domain).power(a=EC('a'), b=EC('b'), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 6/6 [00:01<00:00,  5.94it/s]


,id,a,b,iteration,name,value
0,bef0db4930430383,2,2,0,power,4
1,984d62c949534914,2,3,0,power,8
2,270aa80305613302,2,4,0,power,16
3,5420406315176767,3,2,0,power,9
4,72b9965c74483785,3,3,0,power,27
5,4b4a714552069550,3,4,0,power,81


In [7]:
from batchflow.research import Domain, EC

domain = Domain({'a': [2, 3]}) * Domain({'b': [2, 3, 4]})

research = Research(domain=domain).power(a=EC('a'), b=EC('b'), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 6/6 [00:01<00:00,  5.20it/s]


,id,a,b,iteration,name,value
0,bef0db4930430383,2,2,0,power,4
1,984d62c949534914,2,3,0,power,8
2,270aa80305613302,2,4,0,power,16
3,5420406315176767,3,2,0,power,9
4,72b9965c74483785,3,3,0,power,27
5,4b4a714552069550,3,4,0,power,81


In [8]:
domain = Domain({'a': [2, 3]}) + Domain({'b': [2, 3, 4]})

research = Research(domain=domain).power(kwargs=EC(), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 5/5 [00:01<00:00,  4.50it/s]


,id,a,b,iteration,name,value
0,71d4879f30430383,2,NaN,0,power,8
1,e32d7dce49534914,3,NaN,0,power,27
2,ff973b7605613302,NaN,2,0,power,4
3,6bffa3ee15176767,NaN,3,0,power,8
4,05064ba274483785,NaN,4,0,power,16


In [9]:
from batchflow.research import O

domain = Domain({'a': [2, 3], 'b': [3]})

def inc(x):
    return (x + i for i in range(2))

research = (Research(domain=domain)
            .add_callable(power, a=EC('a'), b=EC('b'))
            .add_generator(inc, save_to='inc', x=O('power'))
           )
research.run(dump_results=False)

research.results.df

0it [00:05, ?it/s]
0it [00:05, ?it/s]
0it [00:05, ?it/s]
0it [00:04, ?it/s]
0it [00:03, ?it/s]
0it [00:01, ?it/s]
0it [00:06, ?it/s]
0it [00:05, ?it/s]
0it [00:05, ?it/s]
0it [00:04, ?it/s]
0it [00:03, ?it/s]
0it [00:01, ?it/s]
0it [00:07, ?it/s]
0it [00:06, ?it/s]
0it [00:06, ?it/s]
0it [00:05, ?it/s]
0it [00:04, ?it/s]
0it [00:02, ?it/s]
100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


,id,a,b,iteration,name,value
0,984d62c930430383,2,3,0,inc,8
1,984d62c930430383,2,3,1,inc,9
2,72b9965c49534914,3,3,0,inc,27
3,72b9965c49534914,3,3,1,inc,28


In [10]:
research = (Research(domain=domain)
            .add_callable(power, a=EC('a'), b=EC('b'))
            .add_generator(inc, save_to='inc', x=O('power'))
            .add_callable('root', power, a=O('inc'), b=0.5, save_to='root',
                          iterations_to_execute='last')
           )
research.run(dump_results=False, n_iters=2)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  8.85it/s]


,id,a,b,iteration,name,value
0,984d62c930430383,2,3,0,inc,8.000000
1,984d62c930430383,2,3,1,inc,9.000000
2,984d62c930430383,2,3,1,root,3.000000
3,72b9965c49534914,3,3,0,inc,27.000000
4,72b9965c49534914,3,3,1,inc,28.000000
5,72b9965c49534914,3,3,1,root,5.291503


In [11]:
research.results.to_df(pivot=True)

,id,a,b,iteration,inc,root
0,984d62c930430383,2,3,0,8,NaN
1,984d62c930430383,2,3,1,9,3.000000
2,72b9965c49534914,3,3,0,27,NaN
3,72b9965c49534914,3,3,1,28,5.291503


In [12]:
class MyCalc:
    def __init__(self, b):
        self.b = b
    
    def power(self, a):
        return a ** self.b

research = (Research(domain=domain)
            .add_instance('calc', MyCalc, b=EC('b'))
            .add_callable('calc.power', a=EC('a'))
            .add_generator(inc, x=O('calc.power'), save_to='inc')
           )

research.run(n_iters=2, dump_results=False)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  7.97it/s]


,id,a,b,iteration,name,value
0,984d62c930430383,2,3,0,inc,8
1,984d62c930430383,2,3,1,inc,9
2,72b9965c49534914,3,3,0,inc,27
3,72b9965c49534914,3,3,1,inc,28


In [15]:
def heavy_callable():
    i = int(2 * 10e1)
    for i in range(i):
        pass

In [13]:
%%timeit

research = Research(n_reps=2).add_callable(heavy_callable)
research.run(dump_results=False)

100%|██████████| 2/2 [00:00<00:00,  3.65it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.37it/s]
0it [00:01, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.27it/s]
0it [00:01, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.27it/s]
0it [00:01, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.54it/s]


785 ms ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit

research = Research(n_reps=2).add_callable(heavy_callable)
research.run(dump_results=False, workers=2)

100%|██████████| 2/2 [00:00<00:00,  4.89it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  4.10it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  4.03it/s]
0it [00:28, ?it/s]
0it [00:01, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  4.59it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  5.50it/s]


654 ms ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
import numpy as np
from batchflow.research import E

def load_data(random):
    return random.normal(size=10)

def mean(array, use_numpy=True):
    if use_numpy:
        return f"{np.mean(array):.02f} (with numpy)"
    return f"{sum(array) / len(array):.02f} (without numpy)"

domain = Domain({'use_numpy': [False, True]})

research = (Research(domain=domain)
            .add_callable(load_data, random=E().random)
            .add_callable(mean, array=O('load_data'), use_numpy=EC('use_numpy'),
                          save_to='stat')
           )

research.remove(ask=False)
research.run(n_iters=2)

research.results.df

0it [00:07, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  8.48it/s]


,id,use_numpy,iteration,name,value
0,7635c7e430430383,False,0,stat,0.03 (without numpy)
1,7635c7e430430383,False,1,stat,0.45 (without numpy)
2,8e95395749534914,True,0,stat,-0.04 (with numpy)
3,8e95395749534914,True,1,stat,0.33 (with numpy)


In [19]:
research = (Research(domain=domain)
            .add_callable(load_data, random=E().random, root=True)
            .add_callable(mean, array=O('load_data'), use_numpy=EC('use_numpy'),
                          save_to='stat')
           )

research.remove(ask=False)
research.run(n_iters=2, branches=2)

research.results.df

100%|██████████| 4/4 [00:00<00:00, 10.22it/s]


,id,use_numpy,iteration,name,value
0,7635c7e430430383,False,0,stat,0.03 (without numpy)
1,7635c7e430430383,False,1,stat,0.45 (without numpy)
2,8e95395749534914,True,0,stat,0.03 (with numpy)
3,8e95395749534914,True,1,stat,0.45 (with numpy)
